<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/continuous_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Continuous Outcomes (Welch t-test)
# 20-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v9.5 §3.7 CFQ/CFS and §4 MeCI
#
# Input: m1, sd1, n1  (group 1), m2, sd2, n2  (group 2)
# Output: p (Welch t p-value), fr (CFS/CFQ), nb (MeCI)
#
# Formulas (exact v9.5):
# - CFS  = ||T| – t*|  → always positive distance to α=0.05 boundary
# - CFQ  = CFS / (1 + CFS)
# - MeCI = D / (1 + D) where D = min(|μ₁-c|,|μ₂-c|) / √(sd₁² + sd₂²)
#         c = (sd₁·μ₂ + sd₂·μ₁)/(sd₁ + sd₂)  weighted midpoint
#
# Note: This calculator requires the standard deviation (SD) of each group, not the standard error of the mean (SE).
# Most papers plot or report mean ± SE. Convert using: SD = SE × √n
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import t

ALPHA = 0.05

# ---------- Core calculations ----------
def continuous_metrics(m1, sd1, n1, m2, sd2, n2):
    if n1 < 2 or n2 < 2 or sd1 <= 0 or sd2 <= 0:
        raise ValueError("Invalid input")

    # Welch t-test
    se1 = sd1 / np.sqrt(n1)
    se2 = sd2 / np.sqrt(n2)
    se_diff = np.sqrt(se1**2 + se2**2)
    t_stat = (m1 - m2) / se_diff
    df = (se1**2 + se2**2)**2 / (se1**4 / (n1-1) + se2**4 / (n2-1))
    p_val = 2 * t.sf(np.abs(t_stat), df)
    t_crit = t.ppf(1 - ALPHA/2, df)

    # CFQ / CFS — symmetric distance to boundary
    cfs = abs(abs(t_stat) - t_crit)
    cfq = cfs / (1 + cfs)

    # MeCI — exact v9.5 Neutrality Boundary Framework (weighted midpoint)
    if sd1 + sd2 == 0:
        meci = 0.0
    else:
        c = (sd1 * m2 + sd2 * m1) / (sd1 + sd2)
        d_num = min(abs(m1 - c), abs(m2 - c))
        d_denom = np.sqrt(sd1**2 + sd2**2)
        D = d_num / d_denom if d_denom > 0 else 0.0
        meci = D / (1 + D)

    return p_val, cfs, cfq, meci

# ---------- High-level ----------
def continuous_outcomes(m1, sd1, n1, m2, sd2, n2):
    p_val, cfs, cfq, meci = continuous_metrics(m1, sd1, n1, m2, sd2, n2)
    return {
        "p": p_val,
        "fr": {"CFS": cfs, "CFQ": cfq},
        "nb": {"MeCI": meci}
    }

# ---------- Narrative ----------
def generate_narrative(res, m1, sd1, n1, m2, sd2, n2):
    diff = abs(m1 - m2)
    se_diff = (sd1**2/n1 + sd2**2/n2)**0.5

    lines = [
        f"Group 1: mean = {m1:.4f}, sd = {sd1:.4f}, n = {n1}",
        f"Group 2: mean = {m2:.4f}, sd = {sd2:.4f}, n = {n2}",
        f"Mean difference = {diff:.4f}  (SE = {se_diff:.4f})",
        "",
        f"p: p-value = {res['p']:.6f} ({'significant' if res['p'] <= ALPHA else 'not significant'})",
        f"fr: CFQ = {res['fr']['CFQ']:.6f} ({'extremely fragile' if res['fr']['CFQ'] < 0.01 else 'very fragile' if res['fr']['CFQ'] < 0.05 else 'fragile' if res['fr']['CFQ'] < 0.10 else 'moderately stable' if res['fr']['CFQ'] < 0.25 else 'very stable'})",
        f"nb: MeCI = {res['nb']['MeCI']:.6f} → {'at neutrality boundary' if res['nb']['MeCI'] < 0.05 else 'near neutrality' if res['nb']['MeCI'] < 0.10 else 'moderately separated' if res['nb']['MeCI'] < 0.25 else 'clearly separated' if res['nb']['MeCI'] < 0.50 else 'far from neutrality'}"
    ]

    if res['p'] <= ALPHA:
        summary = "Claim of difference is supported and is " + \
                  ("very stable and far from neutrality." if res['fr']['CFQ'] > 0.25 and res['nb']['MeCI'] > 0.5 else "fragile and/or close to neutrality.")
    else:
        summary = "Claim of no difference is supported and is " + \
                  ("very stable." if res['fr']['CFQ'] > 0.25 else "fragile (close to significance).")

    lines.append(f"\n{summary}")
    return "\n".join(lines)

# ---------- CLI ----------
def main():
    print("Continuous Outcomes Calculator (Welch t) – v9.5 compliant\n")
    print("Note: this calculator requires standard deviations (SD), not standard errors (SE).")
    print("If only SE is reported, convert using: SD = SE × √n\n")

    print("Group 1")
    m1 = float(input("Mean: "))
    sd1 = float(input("SD: "))
    n1 = int(input("n: "))
    print("\nGroup 2")
    m2 = float(input("Mean: "))
    sd2 = float(input("SD: "))
    n2 = int(input("n: "))

    res = continuous_outcomes(m1, sd1, n1, m2, sd2, n2)

    print("\n================ p–fr–nb ================")
    print(f"p: p-value = {res['p']:.6f} ({'significant' if res['p'] <= ALPHA else 'not significant'})")
    print(f"fr: CFQ = {res['fr']['CFQ']:.6f}")
    print(f"nb: MeCI = {res['nb']['MeCI']:.6f}")
    print("=========================================\n")

    print("Interpretation:")
    print(generate_narrative(res, m1, sd1, n1, m2, sd2, n2))

if __name__ == "__main__":
    main()

Continuous Outcomes Calculator (Welch t) – v9.5 compliant

Note: this calculator requires standard deviations (SD), not standard errors (SE).
If only SE is reported, convert using: SD = SE × √n

Group 1
